In [ ]:
!pip install transformers

In [ ]:
import transformers
import pandas as pd
import torch

In [ ]:
text = pd.read_csv('s3://howardvip-clinical-data/clean_noteevents_text.csv')

In [ ]:
text = text.drop(text.columns[0],axis = 1)
text.columns = ['text']

In [ ]:
text= text.text.to_numpy()

In [ ]:
model = transformers.BertForPreTraining.from_pretrained("bert-base-uncased")

In [ ]:
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
import numpy as np

In [ ]:
sample = np.random.choice(text, 50000)

In [ ]:
import numpy as np

In [ ]:
from tqdm import tqdm

In [ ]:
total = []
for i in tqdm(sample):
    res = None
    length = len(tokenizer.encode(i))
    if length>512:
        while len(i)!=0:
            x = tokenizer(i,truncation = True, max_length = 512,return_tensors='pt')
            out = model.bert(**x).pooler_output.detach().numpy()
            if res is None:
                res = out
            else:
                res = np.append(res,out,axis = 0)
            i = i[512:]
    else:
        x = tokenizer(i,truncation = True, max_length = 512,return_tensors='pt')
        res = (model.bert(**x).pooler_output).detach().numpy()
    if res.shape[0]>1:
        total.append(np.mean(res, axis=0).reshape(1,768))
    else:
        total.append(res)
total = np.array(total)      

In [ ]:
total.shape

In [ ]:
clus_x = total.reshape(500,768)

In [ ]:
clus = KMeans(n_clusters = 4).fit(clus_x)

In [ ]:
labels = clus.predict(clus_x)

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
coords = PCA(2).fit_transform(clus_x)

In [ ]:
import seaborn as sn
import matplotlib.pyplot as plt

In [ ]:
coords = np.vstack((coords.T,labels)).T

In [ ]:
pca_df = pd.DataFrame(coords, columns = ("1st component","2nd component","label"))

In [ ]:
sn.FacetGrid(pca_df, hue="label",height = 8).map(plt.scatter,"1st component","2nd component").add_legend()

In [ ]:
from sklearn.cluster import AgglomerativeClustering

In [ ]:
newclus = AgglomerativeClustering(n_clusters = 4).fit_predict(clus_x)
labels = newclus
coords = PCA(2).fit_transform(clus_x)
coords = np.vstack((coords.T,labels)).T
pca_df = pd.DataFrame(coords, columns = ("1st component","2nd component","label"))
sn.FacetGrid(pca_df, hue="label",height = 8).map(plt.scatter,"1st component","2nd component").add_legend()

In [ ]:
from sklearn.cluster import SpectralClustering

In [ ]:
newclus = SpectralClustering(n_clusters = 8).fit_predict(clus_x)
labels = newclus
coords = PCA(2).fit_transform(clus_x)
coords = np.vstack((coords.T,labels)).T
pca_df = pd.DataFrame(coords, columns = ("1st component","2nd component","label"))
sn.FacetGrid(pca_df, hue="label",height = 8).map(plt.scatter,"1st component","2nd component").add_legend()